# Classificação de dados - MNIST

Este código é parte de tarefa do curso Introdução ao Machine Learning, oferecido pelo Departamento de Matemática da Universidade Federal de Santa Catarina, e ministrado pelo professor Edson Cilos Vargas Júnior. 

A tarefa é a seguinte: criar uma função que desloca a imagem em um píxel em qualquer direção (cima/baixo, esquerda/direita). Então, treinar um modelo de ML num conjunto de teste transladado (em uma direção pra cada instância do conjunto de teste original), e finalmente testar no conjunto de teste separado pelo professor (será obtido fixando a semente aleatória do NumPy). 

In [1]:
import numpy as np
np.random.seed(42)

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1,cache=True)
X,y = mnist['data'],mnist['target']

In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def plotar_numero(digito):
    algum_digito_imagem = digito.reshape(28,28)
    plt.imshow(algum_digito_imagem, cmap=matplotlib.cm.binary,
              interpolation='nearest')
    plt.axis('off')
    plt.show()

In [7]:
# Vamos começar transladando a figura 1 pixel pra cima. 
def transladar_pra_cima(imagem):
    im_transladada = np.append(imagem.reshape(28,28)[1:],[np.zeros(28)],axis=0)
    return np.append([],im_transladada)

# Agora transladar a figura 1 pixel pra baixo. 
def transladar_pra_baixo(imagem):
    im_transladada = np.append([np.zeros(28)],imagem.reshape(28,28)[:-1],axis=0)
    return np.append([],im_transladada)

# Translação 1 pixel à direita
def transladar_pra_direita(imagem):
    im_transladada = np.append([np.zeros(28)],imagem.reshape(28,28)[:,:-1].T,axis=0).T
    return np.append([],im_transladada)

# Translação 1 pixel à esquerda
def transladar_pra_esquerda(imagem):
    im_transladada = np.append(imagem.reshape(28,28)[:,1:].T,[np.zeros(28)],axis=0).T
    return np.append([],im_transladada)

# Finalmente, vamos implementar a translação em qualquer direção: 
def transladar_1_pixel(imagem,direcao):
    if direcao=='c':
        return transladar_pra_cima(imagem)
    if direcao=='b':
        return transladar_pra_baixo(imagem)
    if direcao=='e':
        return transladar_pra_esquerda(imagem)
    if direcao=='d':
        return transladar_pra_direita(imagem)

Agora que temos um jeito prático de transladar imagens em qualquer direção, podemos criar um conjunto de dados de treinamento aumentado. 

In [8]:
# No caso do dataset MNIST, ele já é convenientemente dividido
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Porém vamos embaralhá-lo
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [9]:
# E vamos criar um novo conjunto de treinamento.
X_train_aug = []
y_train_aug = []
for i in range(len(X_train)):
    X_train_aug.append(X_train[i])
    y_train_aug.append(y_train[i])
    for direcao in ['c','b','e','d']:
        X_train_aug.append(transladar_1_pixel(X_train[i],direcao))
        y_train_aug.append(y_train[i])

In [10]:
len(X_train_aug)

300000

Agora temos um conjunto de treinamento com $5\cdot60\,000=300\,000$ instâncias. Vamos treinar um modelo nesse conjunto de treinamento, usando o mesmo algoritmo usado na aula:

In [ ]:
from sklearn.linear_model import SGDClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline #Para criar um pipeline!
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([('std_scaler', StandardScaler()),
                 ('estimator', SGDClassifier(max_iter = 10, random_state=42))
             ]) #O nosso modelo passará pelo std_scaler e depois pelo estimador

#Usaremos 10 epochs, por isso max_iter = 10 
#Cuidado, valores alto de max_iter fazem o algortirmo demorar
#Outros valores para max_iter ou random_state, vão alterar o resultado

param_grid = [{'estimator__loss' : ['hinge', 'log'],
               'estimator__alpha': [1e-4, 1e-2, 1],
               }] #grade de parâmetros para testar

"""É necessário colocar o prefixo estimator__ para indicar que os parâmetros 
serão aplicados ao estimador. Você poderia tentar otimizar o pré-processamento
dentro do pipeline também! Mas observa que muitos parâmetros tornam o processo
bem demora.
"""

#Quanto maior o verbose no GridSearch, mais detalhes sobre o processo
#n_jobs = -1 signifca o número de cores da máquina (-1 usa todos)
grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=10, n_jobs=-1)
grid_search.fit(X_train_aug, y_train_aug)

In [ ]:
model = Pipeline([('std_scaler', StandardScaler()),
                 ('estimator', SGDClassifier(max_iter = 10, random_state=42))
             ])

model.set_params(**grid_search.best_params_) #Introduz no pipeline os parametros

In [ ]:
#Treinando o modelo
model.fit(X_train_aug, y_train_aug)

In [ ]:
#classe que implementa OvO na força
from sklearn.multiclass import OneVsOneClassifier

#Vamos aumentar o número de iterações.
#Lembra que na técnica OvO há mais treinamentos mas pode ser interessante 
#quando o modelo sofre com a escala

model = Pipeline([('std_scaler', StandardScaler()),
                 ('estimator', SGDClassifier(max_iter = 1000, random_state=42))
             ])

model.set_params(**grid_search.best_params_)
ovo_clf = OneVsOneClassifier(model)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(ovo_clf, X_train_aug, y_train_aug, cv=5)
print('Scores da cross-validation oVo:',scores)

In [ ]:
ovo_clf.fit(X_train_aug, y_train_aug)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(ovo_clf, X_train_aug, y_train_aug, cv=5)
print('Scores da validação cruzada:',scores)

In [ ]:
y_pred = ovo_clf.predict(X_test)
conf_mx = confusion_matrix(y_test, y_pred)
print(conf_mx)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)